(problema_mochila)=
# Introducción.

El problema de la mochila es un problema clásico dentro del mundo de la optimización. La idea es que dado una serie de paquetes, con unas ciertas carcaterísticas y un lugar donde se quieren meter esos paquetes (puede ser un contenedor, camión etc), estudiar la manera óptima de colcar los paquetes en el contenedor. 

En este apartado vamos a ver diferentes versiones del problema y la forma de darlo solución.

## versión 1.
(problema_mochila-version_1)=
Tenemos cuatro paquetes, cada paquete tiene un valor, un peso y un volumen. Se trata de maximizar el valor que se introduce teniendo el cuenta que el contenedor tiene un límite de pesos y y de volumen.

En este caso la función objetivo seria:

$$\sum valor_{i}x_{i}$$

Donde $x_i$ es una variable dicotómica que toma el valor 1 si es seleccionado el paquete y cero en caso contrario. Observar que lo que se pretende es maximimizar, luego lo que tenemos que hacer es poner esa función con signo negativo.

Las restricciones se pondrían de forma similar. A continuación, se muestran los valores con los que vamos a trabajar

In [1]:
values = [1, 2, 3, 4] # Valores de los paquetes
weights = [2, 2, 3, 3] #pesos de los paquetes
volumes = [3, 3, 2, 2] # volumen de los paquetes
n = 4 # nº total de paquetes
variables = list(range(n)) # Nos va a servir despues para definir las variables binarias
max_weight = 4 # máximo peso que admite el contenedor
max_volume = 6 # Volumen máximo que admite el contenedor

penalty_strength_weight = 20 # Parámetro de lagrange para la condición de peso
penalty_strength_volume = 20 # parámetro de lagrange para la condición de volumen

A continuación mostramos una tabla de posibles valores y señalamos en negrita la opción que vamos buscando. Observar que hay soluciones que maximizan la función de costes, pero sin embargo no es una solucción factible pues no verifica alguna de las restricciones impuestas.

x_1 | x_2 | x_3 | x_4 | total value | total weight | total volume | feasible|
:----:|:---:|:----:|:----:|:----:|:-----:|:-----:|:------:
1   | 0   |0   | 0   | 1  | 2  | 3  |   True
0   | 1   |0   | 0   | 2  | 2  | 3  |   True
0   | 0   |1   | 0   | 3  | 3  | 2  |   True
**0**   | **0**   |**0**   | **1**   | **4**  | **3**  | **2**  |   **True**
0   | 0   |1   | 1   | 7  | 6  | 4  |   False
1   | 1   |0   | 0   | 3  | 4  | 6  |   True

Comenzamos implementado el modelo de tipo BQM que vamos a utilizar

In [2]:
from dimod import BinaryQuadraticModel

bqm = BinaryQuadraticModel('BINARY')

bqm es una instancia de BinaryQuadraticModel y además le definimos como 'BINARY' para indicar que las variable $x_i$ sólo toman valores cero o uno, como ya se ha comentado antes.

In [3]:
variables = [bqm.add_variable(f'x_{v}', -values[v]) for v in variables]

In [4]:
# Veamos las variables que hemos creado,

In [5]:
variables

['x_0', 'x_1', 'x_2', 'x_3']

observar que a vada variable le hemos dado un ofset o coeficiente con signo negativo, porque lo que queremos es minimizar la función de coste. Veamos a continuación el modelo que hemos creado,

In [6]:
bqm

BinaryQuadraticModel({'x_0': -1.0, 'x_1': -2.0, 'x_2': -3.0, 'x_3': -4.0}, {}, 0.0, 'BINARY')

```{index} bqm.add_linear_inequality_constraint
```
Ahora añadimos la restriccióm de la capacidad de la mochila. Observar que aquí para introducir una restricción de desigualdad utilizamos la propiedad<a href="https://docs.ocean.dwavesys.com/en/latest/docs_dimod/reference/generated/dimod.binary.BinaryQuadraticModel.add_linear_inequality_constraint.html" target="_blank">  bqm.add_linear_inequality_constrain </a>t.

In [7]:
slacks_weight = bqm.add_linear_inequality_constraint(
    [(x, w) for x, w in zip(variables, weights)],
    constant=-max_weight, #este sería el peso máximo permitido
    lagrange_multiplier=penalty_strength_weight, # el multiplicador de lagrange definido al principio
    label='capacity'
)

Hacemos un tratamaiento similar para la restricción del volumen.

In [8]:
slacks_volume = bqm.add_linear_inequality_constraint(
    [(x, v) for x, v in zip(variables, volumes)],
    constant=-max_volume,
    lagrange_multiplier=penalty_strength_volume,
    label='volume_capacity'
)

Para problemas reducidos con por ejemplo menos de 20 variables, podemos resolver el problema QUBO de forma exacta enumerando todas las posibles soluciones

In [9]:
from dimod import ExactSolver

response = ExactSolver().sample(bqm)
samples = response.samples()
best_solution = response.first.sample
print(response.truncate(5))

  slack_capacity_0 slack_capacity_1 slack_capacity_2 ... x_3 energy num_oc.
0                1                0                0 ...   1   -4.0       1
1                0                0                1 ...   1   -4.0       1
2                0                0                0 ...   0   -3.0       1
3                1                0                0 ...   0   -3.0       1
4                0                0                1 ...   0   -3.0       1
['BINARY', 5 rows, 5 samples, 10 variables]


In [10]:
best_solution.items()

dict_items([('slack_capacity_0', 1), ('slack_capacity_1', 0), ('slack_capacity_2', 0), ('slack_volume_capacity_0', 1), ('slack_volume_capacity_1', 0), ('slack_volume_capacity_2', 1), ('x_0', 0), ('x_1', 0), ('x_2', 0), ('x_3', 1)])

Vamos a examinar la solución a continuación.

* Las variables principales especifican el montaje de la mochila.

* El primer conjunto de variables de holgura indica que el peso total de los artículos seleccionados es 1 unidad inferior a la capacidad de peso de la mochila.

* El segundo conjunto de variables de holgura indica que el volumen total de los artículos seleccionados es 1 unidad inferior a la capacidad de volumen de la mochila.

In [11]:
print({k: v for k, v in best_solution.items() if k in variables})
print({k: v for k, v in best_solution.items() if k in {v for v, _ in slacks_weight}})
print({k: v for k, v in best_solution.items() if k in {v for v, _ in slacks_volume}})

{'x_0': 0, 'x_1': 0, 'x_2': 0, 'x_3': 1}
{'slack_capacity_0': 1, 'slack_capacity_1': 0, 'slack_capacity_2': 0}
{'slack_volume_capacity_0': 1, 'slack_volume_capacity_1': 0, 'slack_volume_capacity_2': 1}


Definimos a continuación una función que nos permita saber si se cumplen o no las restricciones del problema

In [13]:
def violations(sample, slack_variables, array, capacity):
    slack_total_weight = 0
    weight = 0
    for i, (v, c) in enumerate(slack_variables):
        slack_total_weight += sample[v] * c
    for i, v in enumerate(variables):
        weight += array[i] * sample[v]
    print('Total of items: ', weight)
    print('Total slack value: ', slack_total_weight)
    print('Left hand side: ', weight + slack_total_weight)
    print('Satisfied? ', weight + slack_total_weight <= capacity)

In [14]:
print('Weight capacity constraint:')
violations(samples[0], slacks_weight, weights, max_weight)
print('Volume capacity constraint:')
violations(samples[0], slacks_volume, volumes, max_volume)

Weight capacity constraint:
Total of items:  3
Total slack value:  1
Left hand side:  4
Satisfied?  True
Volume capacity constraint:
Total of items:  2
Total slack value:  4
Left hand side:  6
Satisfied?  True


## Version 2 (con CQM). 
(problema_mochila-version_2)=

El problema es similar al mostrado anteriormente, pero en esta ocasión utilizo una instancia de la clase CQM. Este tipo de solución también lo podemos ver en <a herf="https://github.com/dwave-examples/knapsack" target="_blank"> https://github.com/dwave-examples/knapsack </a>.

In [15]:
from dimod import Binary, CQM, quicksum
from dwave.system import LeapHybridCQMSampler
import itertools

In [16]:
values = [34, 25,78,21,64]
weights = [3,5,9,4,2]
W = 10
n= len(values)

In [17]:
 #Creamos las variable binarias
x = [Binary(i)  for i in range(n)]

In [18]:
 #Creamos las variable binarias
x = [Binary(i)  for i in range(n)]

In [21]:

 cqm = CQM()
# Añadimos la función objetivo
cqm.set_objective(quicksum(-values[i]*x[i] for i in range(n)))

In [22]:
# Añadimos las restricciones
cqm.add_constraint(quicksum(weights[i]*x[i] for i in range(n)) <= W, label ='peso maximo')
cqm.add_constraint(quicksum(x[i] for i in range(n))<=2, label ='items máximo ')

'items máximo '

In [23]:
# submitimos el sampler
sampler =  LeapHybridCQMSampler()
sampleset = sampler.sample_cqm(cqm)

## Versión 3 (Con otras restricciones).
(problema_mochila-version_3)=

En este problema se trata de empaquetar un camión de reparto. En este ejemplo, consideramos un escenario simple en el que tenemos un solo camión y un solo tamaño de caja. Queremos elegir paquetes para cargar en el camión de modo que se maximice la cantidad de paquetes con estado de envío prioritario y se minimice el tiempo de tránsito total. No se debe exceder la capacidad de peso y tamaño del camión.

In [24]:
from dimod import ConstrainedQuadraticModel, Binaries, quicksum 
from dwave.system import LeapHybridCQMSampler 
import random 
import numpy as np

In [25]:
# Problem set up
num_packages = 300

# Prioridad de cada paquete, 3 = high priority, 1 = low priority
priority = random.choices((1, 2, 3), k=num_packages)

# Número de días transcurridos desde que se encargó cada paquete (a más días, mayor prioridad)
days_since_order = random.choices((0, 1, 2, 3), k=num_packages)

# Weight of each package
weight = random.choices(range(1, 101), k=num_packages)

# Peso máximo y número de bultos que puede transportar el camión
max_weight = 3000  
max_packages = 100

# Pesos para cada sumando de la función objetivo
obj_weight_priority = 1.0 
obj_weight_days = 1.0

In [26]:
# Build the CQM
cqm = ConstrainedQuadraticModel()

# Create the binary variables
bin_variables = list(Binaries(range(num_packages)))

In [27]:
# Veamos cómo es una variable
bin_variables[0]

BinaryQuadraticModel({0: 1.0}, {}, 0.0, 'BINARY')

In [28]:
# quicksum() se utiliza para indicar que es una suma

# ----------------- Objective functions ----------------- 

# 1. Maximize priority shipping packages
objective1 = -obj_weight_priority * quicksum(priority[i] * bin_variables[i] 
                for i in range(num_packages))

# 2. Minimize customers wait time
objective2 = -obj_weight_days * quicksum(days_since_order[i] * bin_variables[i]
                for i in range(num_packages))

# Add the objectives to the CQM
cqm.set_objective(objective1 + objective2)

In [29]:
# ----------------- Constraints ----------------- 

# Add the maximum capacity constraint
cqm.add_constraint(quicksum(weight[i] * bin_variables[i] for i in
range(num_packages)) <= max_weight, label='max_weight')

# Add the maximum package (or truck size) constraint
cqm.add_constraint(quicksum(bin_variables[i] for i in range(num_packages)) 
                    == max_packages, label='max_packages')

'max_packages'

In [30]:
# -----------------  Submit to the CQM sampler -----------------
cqm_sampler = LeapHybridCQMSampler() 
sampleset = cqm_sampler.sample_cqm(cqm, label='Truck Packing Demo')

In [31]:
feasible_sampleset = sampleset.filter(lambda d: d.is_feasible)

if not len(feasible_sampleset): 
    print("\nNo feasible solution found.\n")

else: 
    first_feasible_sol = feasible_sampleset.first.sample
    print(first_feasible_sol)

{0: 1.0, 1: 0.0, 2: 1.0, 3: 1.0, 4: 0.0, 5: 0.0, 6: 1.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 1.0, 11: 0.0, 12: 1.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 1.0, 21: 1.0, 22: 1.0, 23: 0.0, 24: 0.0, 25: 0.0, 26: 0.0, 27: 0.0, 28: 1.0, 29: 0.0, 30: 0.0, 31: 0.0, 32: 0.0, 33: 1.0, 34: 1.0, 35: 0.0, 36: 0.0, 37: 0.0, 38: 0.0, 39: 1.0, 40: 0.0, 41: 0.0, 42: 0.0, 43: 1.0, 44: 0.0, 45: 0.0, 46: 0.0, 47: 1.0, 48: 0.0, 49: 0.0, 50: 1.0, 51: 1.0, 52: 1.0, 53: 0.0, 54: 0.0, 55: 0.0, 56: 0.0, 57: 1.0, 58: 1.0, 59: 0.0, 60: 1.0, 61: 0.0, 62: 1.0, 63: 1.0, 64: 0.0, 65: 0.0, 66: 0.0, 67: 0.0, 68: 0.0, 69: 1.0, 70: 0.0, 71: 1.0, 72: 0.0, 73: 0.0, 74: 0.0, 75: 0.0, 76: 0.0, 77: 1.0, 78: 1.0, 79: 0.0, 80: 0.0, 81: 0.0, 82: 0.0, 83: 0.0, 84: 0.0, 85: 0.0, 86: 0.0, 87: 0.0, 88: 0.0, 89: 1.0, 90: 0.0, 91: 0.0, 92: 1.0, 93: 0.0, 94: 0.0, 95: 0.0, 96: 0.0, 97: 0.0, 98: 0.0, 99: 0.0, 100: 0.0, 101: 1.0, 102: 0.0, 103: 0.0, 104: 0.0, 105: 0.0, 106: 1.0, 107: 0.0, 108: 0.0, 109: 0.0, 110: 0.0,

In [32]:
# ----------------- Process the results -----------------
feasible_sampleset = sampleset.filter(lambda d: d.is_feasible)

if not len(feasible_sampleset): 
    print("\nNo feasible solution found.\n")

else: 
    first_feasible_sol = feasible_sampleset.first.sample

    # Characterize the problem
    problem_array = np.zeros((3, 4)).astype(int) 
    for i in range(num_packages):
        problem_array[-1 * (priority[i]-3)][-1 * (days_since_order[i] - 3)] += 1

    print("\n****************** PROBLEM ******************\n") 
    print('Days since order was placed')
    print('{:>5s}{:>5s}{:>5s}{:>5s}{:>5s}'.format('Priority |',
             '3', '2', '1', '0')) 
    print('-' * 40)

    for i in range(3): 
        print('{:>5s}{:>10s}{:>5s}{:>5s}{:>5s}'.format(str(-1*(i- 3)), 
                str(problem_array[i][0]), str(problem_array[i][1]),
                str(problem_array[i][2]), str(problem_array[i][3])))

    # Calculate number of packages with each priority and number of days since
    # order in the solution
    chosen = [i for i in first_feasible_sol if first_feasible_sol[i] == 1]
    total_weight = quicksum(weight[i] for i in chosen) 
    solution_priorities = [priority[i] for i in chosen] 
    solution_days_since_order = [days_since_order[i] for i in chosen]

    # Characterize the solution 
    # Packages with higher priority (upper row) and the most days since the
    # order (left most column) should be prioritized in the selection
    results_array = np.zeros((3, 4)).astype(int) 
    for i in chosen:
        results_array[-1 * (priority[i]-3)][-1 * (days_since_order[i] - 3)] += 1

    print("\n****************** SOLUTION ******************\n") 
    print('Days since order was placed')
    print('{:>5s}{:>5s}{:>5s}{:>5s}{:>5s}'.format('Priority |',
            '3', '2', '1', '0')) 
    print('-' * 40)

    for i in range(3): 
        print('{:>5s}{:>10s}{:>5s}{:>5s}{:>5s}'.format(str(-1*(i - 3)),
                str(results_array[i][0]), str(results_array[i][1]),
                str(results_array[i][2]), str(results_array[i][3])))

    print(("\nTotal number of selected items: {}".format(len(chosen))))
    print("Total weight of selected items: {}".format(total_weight))


****************** PROBLEM ******************

Days since order was placed
Priority |    3    2    1    0
----------------------------------------
    3        23   19   28   31
    2        23   20   21   27
    1        23   29   34   22

****************** SOLUTION ******************

Days since order was placed
Priority |    3    2    1    0
----------------------------------------
    3        23   15   10    8
    2        14    4    5    0
    1        18    3    0    0

Total number of selected items: 100
Total weight of selected items: 3000
